<a href="https://colab.research.google.com/github/mssamoilenko/AnlDataPract/blob/main/PraktAnl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практична робота №3
## Тема: класифікація за допомогою логістичної
## регресії та наївного Баєса.
## Мета: закріплення навичок роботи із категорійними даними, їх по
## передньої обробки та класифікація даних.
## Завдання
## 1.    Використати дані відповідно до власного варіанта (в мене це варіант №2). Вивести таблицю за допомогою Pandas.
## 2.   Визначити числові, категорійні впорядковані, категорійні невпорядковані
## змінні. Для кожної категорійної змінної написати код, який виводить всі
## унікальні значення, котрі вона набуває в наборі даних.
## 3.   Закодувати категорійні впорядковані числами 0, 1, 2… (або іншими
## числами аналогічним чином), до невпорядкованих застосувати опорне
## кодування.
## 4.   Застосувати методи логістичної регресії та наївного Баєса до оброблених
## даних для передбачення змінної «Статус проєкта». На крос-валідації обрати
## кращу модель.
## Варіант роботи № 2
## Хід роботи

## 1.   Виводжу таблицю з даними відповідно моєму варіанту.







In [1]:
import pandas as pd
df = pd.read_csv("2.csv")
df

,Потужність (л/г),Тара,Якість,Структура,Відстань від джерела (км),Матеріал,Вартість (грн),Статус проєкта
0,38.33,Особливе замовлення,Низька,Ромбовидна,2479.88,Скло,954.65,Заблоковано
1,76.99,Особливе замовлення,Дуже низька,Фрактальна,1825.69,Залізо,1216.99,Узгоджено
2,48.72,Каністра,Дуже низька,Решітка,2034.32,Залізо,861.93,Узгоджено
3,63.23,Своя,Низька,Ромбовидна,2518.65,Поліпропілен,1055.64,Заблоковано
4,24.63,Бочка,Дуже низька,Фрактальна,2661.19,Скло,1063.26,Заблоковано
...,...,...,...,...,...,...,...,...
4995,37.36,Бочка,Висока,Решітка,2462.35,Льон,911.51,Заблоковано
4996,41.68,Бочка,Висока,Фрактальна,2774.46,Залізо,1136.20,Заблоковано
4997,51.92,Каністра,Середня,Ромбовидна,2482.63,Льон,1056.54,Узгоджено
4998,49.98,Каністра,Середня,Фрактальна,1768.00,Поліпропілен,1134.95,Узгоджено


## 2. Визначаю числові, категорійні впорядковані, категорійні невпорядковані змінні. Для кожної категорійної змінної пишу код, який виводить всі унікальні значення, котрі вона набуває в наборі даних.

In [3]:
# 1. Числові змінні
numerical_vars = ['Потужність (л/г)', 'Відстань від джерела (км)', 'Вартість (грн)']

In [4]:
# 2. Категорійні впорядковані змінні
ordinal_vars = ['Якість']

In [5]:
# 3. Категорійні невпорядковані змінні
categorical_vars = ['Тара', 'Структура', 'Матеріал', 'Статус проєкта']

In [7]:
df['Якість'].unique()

array(['Низька', 'Дуже низька', 'Середня', 'Висока'], dtype=object)

In [8]:
df['Тара'].unique()

array(['Особливе замовлення', 'Каністра', 'Своя', 'Бочка'], dtype=object)

In [9]:
df['Структура'].unique()

array(['Ромбовидна', 'Фрактальна', 'Решітка'], dtype=object)

In [10]:
df['Матеріал'].unique()

array(['Скло', 'Залізо', 'Поліпропілен', 'Льон'], dtype=object)

In [11]:
df['Статус проєкта'].unique()

array(['Заблоковано', 'Узгоджено'], dtype=object)

3. Закодувати категорійні впорядковані числами 0, 1, 2… (або іншими
числами аналогічним чином), до невпорядкованих застосувати опорне
кодування.